In [187]:
import sys
import os
# Get the current directory and parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, current_dir)

# Import from the-machine module
import importlib.util
the_machine_path = os.path.join(parent_dir, "the-machine")
sys.path.insert(0, the_machine_path)

# Import all from the-machine and custom_matroid_functions
from HM_module import *
from custom_matroid_functions import *

In [188]:
P = LaurentPolynomialRing(ZZ, 'x')
x = P.gen()
k, n = 4, 5
M = matroids.CompleteGraphic(n)
#M = matroids.Uniform(k, n)
L = M.lattice_of_flats()
r = M.rank()
rank_func = lambda x : M.rank(x)

#e = next(x for x in M.groundset() if x  not in M.coloops())
e = list(M.groundset())[0]
print(M.groundset())
print("deleting ", e)
mdele = M.delete(e)
Eq_M = create_deformed_algebra(M, base_ring=P)
Eq_mdele = create_deformed_algebra(mdele, base_ring=P)
E_mdele = Eq_mdele.natural()

E = Eq_M.natural()
E_basis = E.basis()
E_mdele_basis = E_mdele.basis()

C = Eq_M.C()
C_dele = Eq_mdele.C()
C_basis = C.basis()
C_dele_basis = C_dele.basis()

Z = Eq_M.zeta()
Z_dele = Eq_mdele.zeta()
Z_basis = Z.basis()

flats_keys = list(E_basis.keys())

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})
deleting  0


In [189]:
print("rank of M: ", r)
print("chow polynomial of M: ", chow_polynomial(M))
print(reduced_characteristic_polynomial(M))
C(E(flats_keys[-1]))

rank of M:  4
chow polynomial of M:  x^3 + 41*x^2 + 41*x + 1
x^3 - 9*x^2 + 26*x - 24


-(x^-2-9+26*x^2-24*x^4)*C[frozenset()] - (x^-1-5*x+6*x^3)*C[frozenset({0})] - (1-2*x^2)*C[frozenset({0, 9})] - x*C[frozenset({0, 1, 4, 9})] - x*C[frozenset({0, 8, 9, 7})] - x*C[frozenset({0, 2, 3, 5, 6, 9})] - (1-2*x^2)*C[frozenset({0, 8})] - (1-2*x^2)*C[frozenset({0, 2, 5})] - x*C[frozenset({0, 8, 2, 5})] - x*C[frozenset({0, 1, 3, 4, 6, 8})] - (1-2*x^2)*C[frozenset({0, 7})] - (1-2*x^2)*C[frozenset({0, 3, 6})] - x*C[frozenset({0, 3, 6, 7})] - (1-2*x^2)*C[frozenset({0, 1, 4})] - x*C[frozenset({0, 1, 2, 4, 5, 7})] - (x^-1-5*x+6*x^3)*C[frozenset({6})] - (1-2*x^2)*C[frozenset({6, 7})] - x*C[frozenset({4, 5, 6, 7, 8, 9})] - x*C[frozenset({1, 2, 6, 7})] - (1-2*x^2)*C[frozenset({9, 5, 6})] - x*C[frozenset({1, 5, 6, 9})] - (1-2*x^2)*C[frozenset({8, 4, 6})] - (1-2*x^2)*C[frozenset({2, 6})] - x*C[frozenset({8, 2, 4, 6})] - (1-2*x^2)*C[frozenset({1, 6})] - (x^-1-5*x+6*x^3)*C[frozenset({3})] - (1-2*x^2)*C[frozenset({3, 5})] - x*C[frozenset({3, 4, 5, 7})] - x*C[frozenset({8, 1, 3, 5})] - (1-2*x^2)*

In [190]:
rev = {value: key for key, value in E_basis.items()}

def delta_flat(F):
    F_del_e = frozenset(set(F) - {e})
    return Eq_mdele(x^( - M.rank(F) + mdele.rank(F_del_e)) * E_mdele_basis[F_del_e])
    
def delta(elem):
    rep = E(elem)
    decom = {monomial: coeff for monomial, coeff in zip(rep.monomials(), rep.coefficients())}
    return Eq_mdele(sum(decom[F] * delta_flat(rev[F]) for F in decom))

dict = {F : C_dele(delta(C[F])) for F in E_basis.keys()}

In [191]:
def flat_division(M, e):
    L = M.lattice_of_flats()
    flat_div = {'S':set(), 'T':set(), 'A':set(), 'B':set()}
    for F in L:
        if e in F:
            if F - {e} in L:
                flat_div['T'].add(F)
            else:
                flat_div['A'].add(F)
        else:
            if F.union({e}) in L:
                flat_div['S'].add(F)
            else:
                flat_div['B'].add(F)
    return flat_div

In [192]:
from IPython.display import HTML, display
import pandas as pd

# Get the flat division
flat_div = flat_division(M, e)

# Create separate data lists for each category
data_s = []
data_t = []
data_a = []
data_b = []

basis_list_mdele = list(mdele.lattice_of_flats())

for F in dict:
    elem = dict[F]
    coeff_empty = elem.monomial_coefficients().get(frozenset(), 0)
    
    terms = []
    for S, coeff in elem.monomial_coefficients().items():
        terms.append(f"({coeff})*C[{str(set(S))}]")
    formatted_expr = " + ".join(terms) if terms else "0"
    
    row_data = {
        'F': str(set(F)),
        'rk(F)': M.rank(F),
        'delta(C[F])': formatted_expr,
        'coeff of empty': coeff_empty,
        'in zeta basis': str(Z_dele(elem)),
        'in natural basis': str(E_mdele(elem))
    }
    
    # Categorize the flat and add to appropriate list
    if F in flat_div['S']:
        data_s.append(row_data)
    elif F in flat_div['T']:
        data_t.append(row_data)
    elif F in flat_div['A']:
        data_a.append(row_data)
    elif F in flat_div['B']:
        data_b.append(row_data)

# Create and display tables for each category
def display_table(data, category_name, color='lightblue'):
    if data:
        df = pd.DataFrame(data)
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} ({len(data)} flats)</h3>'))
        display(HTML(
            f'<div style="height: 300px; overflow-y: auto; border: 2px solid {color}; padding: 10px;">{df.to_html(index=False)}</div>'
        ))
    else:
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} (0 flats)</h3>'))
        display(HTML('<p>No flats in this category.</p>'))

# Display all four tables
display_table(data_s, 'S', 'green')
display_table(data_t, 'T', 'coral') 
display_table(data_a, 'A', 'blue')
display_table(data_b, 'B', 'red')

F,rk(F),delta(C[F]),coeff of empty,in zeta basis,in natural basis
set(),0,(1)*C[set()],1,Z[frozenset()],E[frozenset()]
{8},1,(1)*C[{8}],0,Z[frozenset({8})],x*E[frozenset()] + E[frozenset({8})]
{9},1,(1)*C[{9}],0,Z[frozenset({9})],x*E[frozenset()] + E[frozenset({9})]
{7},1,(1)*C[{7}],0,Z[frozenset({7})],x*E[frozenset()] + E[frozenset({7})]
"{8, 9, 7}",2,"(1)*C[{8, 9, 7}]",0,"Z[frozenset()] + Z[frozenset({8, 9, 7})]","(1+x^2)*E[frozenset()] + x*E[frozenset({8})] + x*E[frozenset({9})] + x*E[frozenset({7})] + E[frozenset({8, 9, 7})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis,in natural basis
{0},1,(x^-1 + x)*C[set()],x^-1 + x,(x^-1+x)*Z[frozenset()],(x^-1+x)*E[frozenset()]
"{0, 7}",2,(1)*C[set()] + (x^-1 + x)*C[{7}],1,Z[frozenset()] + (x^-1+x)*Z[frozenset({7})],(2+x^2)*E[frozenset()] + (x^-1+x)*E[frozenset({7})]
"{0, 8}",2,(1)*C[set()] + (x^-1 + x)*C[{8}],1,Z[frozenset()] + (x^-1+x)*Z[frozenset({8})],(2+x^2)*E[frozenset()] + (x^-1+x)*E[frozenset({8})]
"{0, 9}",2,(1)*C[set()] + (x^-1 + x)*C[{9}],1,Z[frozenset()] + (x^-1+x)*Z[frozenset({9})],(2+x^2)*E[frozenset()] + (x^-1+x)*E[frozenset({9})]
"{0, 8, 9, 7}",3,"(x^-1 + x)*C[set()] + (1)*C[{8}] + (1)*C[{9}] + (1)*C[{7}] + (x^-1 + x)*C[{8, 9, 7}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + Z[frozenset({8})] + Z[frozenset({9})] + Z[frozenset({7})] + (x^-1+x)*Z[frozenset({8, 9, 7})]","(2*x^-1+6*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({8})] + (2+x^2)*E[frozenset({9})] + (2+x^2)*E[frozenset({7})] + (x^-1+x)*E[frozenset({8, 9, 7})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis,in natural basis
"{0, 1, 4}",2,"(1)*C[set()] + (1)*C[{1, 4}]",1,"2*Z[frozenset()] + Z[frozenset({1, 4})]","(2+x^2)*E[frozenset()] + x*E[frozenset({1})] + x*E[frozenset({4})] + E[frozenset({1, 4})]"
"{0, 3, 6}",2,"(1)*C[set()] + (1)*C[{3, 6}]",1,"2*Z[frozenset()] + Z[frozenset({3, 6})]","(2+x^2)*E[frozenset()] + x*E[frozenset({3})] + x*E[frozenset({6})] + E[frozenset({3, 6})]"
"{0, 2, 5}",2,"(1)*C[set()] + (1)*C[{2, 5}]",1,"2*Z[frozenset()] + Z[frozenset({2, 5})]","(2+x^2)*E[frozenset()] + x*E[frozenset({5})] + x*E[frozenset({2})] + E[frozenset({2, 5})]"
"{0, 1, 2, 4, 5, 7}",3,"(x^-1 + x)*C[set()] + (1)*C[{7}] + (1)*C[{1, 2, 4, 5, 7}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({7})] + Z[frozenset({1})] + Z[frozenset({5})] + Z[frozenset({4})] + Z[frozenset({2})] + Z[frozenset({1, 2, 4, 5, 7})]","(2*x^-1+9*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({7})] + (1+x^2)*E[frozenset({1})] + (1+x^2)*E[frozenset({5})] + x*E[frozenset({1, 5})] + (1+x^2)*E[frozenset({4})] + x*E[frozenset({1, 4})] + (1+x^2)*E[frozenset({2})] + x*E[frozenset({2, 5})] + x*E[frozenset({2, 4})] + x*E[frozenset({1, 2, 7})] + x*E[frozenset({4, 5, 7})] + E[frozenset({1, 2, 4, 5, 7})]"
"{0, 3, 6, 7}",3,"(x^-1 + x)*C[set()] + (1)*C[{7}] + (1)*C[{3, 6, 7}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({7})] + Z[frozenset({3})] + Z[frozenset({6})] + Z[frozenset({3, 6, 7})]","(2*x^-1+6*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({7})] + (1+x^2)*E[frozenset({3})] + x*E[frozenset({3, 7})] + (1+x^2)*E[frozenset({6})] + x*E[frozenset({3, 6})] + x*E[frozenset({6, 7})] + E[frozenset({3, 6, 7})]"
"{0, 1, 3, 4, 6, 8}",3,"(x^-1 + x)*C[set()] + (1)*C[{8}] + (1)*C[{1, 3, 4, 6, 8}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({8})] + Z[frozenset({1})] + Z[frozenset({4})] + Z[frozenset({3})] + Z[frozenset({6})] + Z[frozenset({1, 3, 4, 6, 8})]","(2*x^-1+9*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({8})] + (1+x^2)*E[frozenset({1})] + (1+x^2)*E[frozenset({4})] + x*E[frozenset({1, 4})] + (1+x^2)*E[frozenset({3})] + x*E[frozenset({8, 1, 3})] + x*E[frozenset({3, 4})] + (1+x^2)*E[frozenset({6})] + x*E[frozenset({1, 6})] + x*E[frozenset({3, 6})] + x*E[frozenset({8, 4, 6})] + E[frozenset({1, 3, 4, 6, 8})]"
"{0, 8, 2, 5}",3,"(x^-1 + x)*C[set()] + (1)*C[{8}] + (1)*C[{8, 2, 5}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({8})] + Z[frozenset({5})] + Z[frozenset({2})] + Z[frozenset({8, 2, 5})]","(2*x^-1+6*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({8})] + (1+x^2)*E[frozenset({5})] + x*E[frozenset({8, 5})] + (1+x^2)*E[frozenset({2})] + x*E[frozenset({8, 2})] + x*E[frozenset({2, 5})] + E[frozenset({8, 2, 5})]"
"{0, 2, 3, 5, 6, 9}",3,"(x^-1 + x)*C[set()] + (1)*C[{9}] + (1)*C[{2, 3, 5, 6, 9}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({9})] + Z[frozenset({5})] + Z[frozenset({2})] + Z[frozenset({3})] + Z[frozenset({6})] + Z[frozenset({2, 3, 5, 6, 9})]","(2*x^-1+9*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({9})] + (1+x^2)*E[frozenset({5})] + (1+x^2)*E[frozenset({2})] + x*E[frozenset({2, 5})] + (1+x^2)*E[frozenset({3})] + x*E[frozenset({9, 2, 3})] + x*E[frozenset({3, 5})] + (1+x^2)*E[frozenset({6})] + x*E[frozenset({2, 6})] + x*E[frozenset({3, 6})] + x*E[frozenset({9, 5, 6})] + E[frozenset({2, 3, 5, 6, 9})]"
"{0, 1, 4, 9}",3,"(x^-1 + x)*C[set()] + (1)*C[{9}] + (1)*C[{1, 4, 9}]",x^-1 + x,"(2*x^-1+2*x)*Z[frozenset()] + 2*Z[frozenset({9})] + Z[frozenset({1})] + Z[frozenset({4})] + Z[frozenset({1, 4, 9})]","(2*x^-1+6*x+x^3)*E[frozenset()] + (2+x^2)*E[frozenset({9})] + (1+x^2)*E[frozenset({1})] + x*E[frozenset({1, 9})] + (1+x^2)*E[frozenset({4})] + x*E[frozenset({1, 4})] + x*E[frozenset({9, 4})] + E[frozenset({1, 4, 9})]"
"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}",4,"(x^-2 + 8 + x^2)*C[set()] + (x^-1 + x)*C[{8}] + (x^-1 + x)*C[{9}] + (x^-1 + x)*C[{7}] + (1)*C[{8, 9, 7}] + (1)*C[{1, 2, 3, 4, 5, 6, 7, 8, 9}]",x^-2 + 8 + x^2,"(2*x^-2+37+2*x^2)*Z[frozenset()] + (2*x^-1+2*x)*Z[frozenset({8})] + (2*x^-1+2*x)*Z[frozense

F,rk(F),delta(C[F]),coeff of empty,in zeta basis,in natural basis
{1},1,(1)*C[{1}],0,Z[frozenset({1})],x*E[frozenset()] + E[frozenset({1})]
"{1, 9}",2,"(1)*C[{1, 9}]",0,"Z[frozenset()] + Z[frozenset({1, 9})]","(1+x^2)*E[frozenset()] + x*E[frozenset({1})] + x*E[frozenset({9})] + E[frozenset({1, 9})]"
{5},1,(1)*C[{5}],0,Z[frozenset({5})],x*E[frozenset()] + E[frozenset({5})]
"{1, 5}",2,"(1)*C[{1, 5}]",0,"Z[frozenset()] + Z[frozenset({1, 5})]","(1+x^2)*E[frozenset()] + x*E[frozenset({1})] + x*E[frozenset({5})] + E[frozenset({1, 5})]"
"{8, 5}",2,"(1)*C[{8, 5}]",0,"Z[frozenset()] + Z[frozenset({8, 5})]","(1+x^2)*E[frozenset()] + x*E[frozenset({8})] + x*E[frozenset({5})] + E[frozenset({8, 5})]"
{4},1,(1)*C[{4}],0,Z[frozenset({4})],x*E[frozenset()] + E[frozenset({4})]
"{9, 4}",2,"(1)*C[{9, 4}]",0,"Z[frozenset()] + Z[frozenset({9, 4})]","(1+x^2)*E[frozenset()] + x*E[frozenset({9})] + x*E[frozenset({4})] + E[frozenset({9, 4})]"
{2},1,(1)*C[{2}],0,Z[frozenset({2})],x*E[frozenset()] + E[frozenset({2})]
"{8, 2}",2,"(1)*C[{8, 2}]",0,"Z[frozenset()] + Z[frozenset({8, 2})]","(1+x^2)*E[frozenset()] + x*E[frozenset({8})] + x*E[frozenset({2})] + E[frozenset({8, 2})]"
"{2, 4}",2,"(1)*C[{2, 4}]",0,"Z[frozenset()] + Z[frozenset({2, 4})]","(1+x^2)*E[frozenset()] + x*E[frozenset({4})] + x*E[frozenset({2})] + E[frozenset({2, 4})]"


In [193]:
#\Delta(C[F]) - C[F\e] = 
def delta_diff(F):
    return delta(C(F)) -C_dele(F - {e})

print(E_basis.keys()[-1])
C_dele(delta_diff(E_basis.keys()[-1]))


frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})


(x^-2+8+x^2)*C[frozenset()] + (x^-1+x)*C[frozenset({8})] + (x^-1+x)*C[frozenset({9})] + (x^-1+x)*C[frozenset({7})] + C[frozenset({8, 9, 7})]

In [194]:
def delta_diff_comp(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly1 = f(M.contract(G))(x^-2)
        poly2 = f(M.contract(G.union({e})))(x^-2)
        poly3 = f(mdele.contract(G))(x^-2)
        poly_coeff = P(x^(M.rank(ground_set) - M.rank(G)) * (poly1 + x^(-2) * poly2 - poly3))
        sum += Eq_mdele(poly_coeff * E_mdele_basis[G])
    return C_dele(sum)

delta_diff_comp(M, chow_polynomial)

(x^-2+8+x^2)*C[frozenset()] + (x^-1+x)*C[frozenset({9})] + (x^-1+x)*C[frozenset({8})] + (x^-1+x)*C[frozenset({7})] + C[frozenset({8, 9, 7})]

In [195]:
def delta_diff_comp_2tutte(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly = x^(M.rank(ground_set) - M.rank(G)) * (-1+x^-2) * f(M.contract(G.union({e})))(x^-2)
        sum += Eq_mdele(P(poly) * E_mdele_basis[G])
    return C_dele(sum)

delta_diff_comp_2tutte(M, characteristic_polynomial)

(x^-4-11*x^-2+33-37*x^2+14*x^4)*C[frozenset()] + (x^-3-5*x^-1+7*x-3*x^3)*C[frozenset({9})] + (x^-3-5*x^-1+7*x-3*x^3)*C[frozenset({8})] + (x^-3-5*x^-1+7*x-3*x^3)*C[frozenset({7})] + (x^-2-2+x^2)*C[frozenset({8, 9, 7})]

In [196]:
def delta_diff_comp_chow(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly1 = x^(-M.rank(ground_set) + M.rank(G) + 2)
        poly2 = f(M.contract(G))(x^2)
        poly3 = f(M.contract(G.union({e})))(x^2)
        poly4 = f(mdele.contract(G))(x^2)
        poly_coeff = P(poly1 * (poly2 + poly3 - poly4))
        term = C_dele(poly_coeff * E_mdele_basis[G])
        sum += term
    return C_dele(sum)

diff = delta_diff_comp_chow(M, chow_polynomial)
print(diff)

(x^-2+8+x^2)*C[frozenset()] + (x^-1+x)*C[frozenset({7})] + (x^-1+x)*C[frozenset({9})] + (x^-1+x)*C[frozenset({8})] + C[frozenset({8, 9, 7})]


In [197]:
def delta_diff_chow_conj(M, f):
    ground_set = M.groundset()  
    sum = 0
    for G in flat_div['S']:
        poly1 = x^(M.rank(ground_set) - M.rank(G) - 2)
        poly2 = f(M.contract(G.union({e})))(x^-2)
        sum += Eq_mdele(P(poly1 * poly2) * C_dele_basis[G])
    return C_dele(sum)

delta_diff_chow_conj(M, chow_polynomial)


(x^-2+8+x^2)*C[frozenset()] + (x^-1+x)*C[frozenset({9})] + (x^-1+x)*C[frozenset({8})] + (x^-1+x)*C[frozenset({7})] + C[frozenset({8, 9, 7})]

In [198]:
delta_diff_chow_conj(M, chow_polynomial) == delta_diff(flats_keys[-1])

True

In [199]:
chi = reduced_characteristic_polynomial
flat_div['S']

{frozenset(),
 frozenset({9}),
 frozenset({8}),
 frozenset({7}),
 frozenset({7, 8, 9})}

In [200]:
def delta_diff_chow_conj_2(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)
    sum = 0
    for F in flat_div['S']:
        print("iterating over", F)
        print("G in L.order_ideal([F]):", L.order_ideal([F]))
        res = M.delete(ground_set - set(F))
        sumF = 0
        for G in L.order_ideal([F]):
            chibar = chi(res.contract(G))(x^-2)
            poly1 = x^(M.rank(ground_set) - M.rank(G))
            poly2 = f(M.contract(F))(x^-2)*chibar
            poly3 = f(M.contract(F.union({e})))(x^-2)*chibar
            poly4 = f(mdele.contract(F))(x^-2)*chibar
            term = P(poly1 * (poly2 + x^(-2)*poly3 - poly4))
            print(G, term)
            sumF += - C_dele(term * C_dele_basis[G])
        print("summing over", F, "gives", sumF)
        sum += sumF
    return sum

delta_diff_chow_conj_2(M, chow_polynomial)

iterating over frozenset({9})
G in L.order_ideal([F]): [frozenset(), frozenset({9})]
frozenset() 1 + 2*x^2
frozenset({9}) -x^-1 - 2*x
summing over frozenset({9}) gives -(1+2*x^2)*C[frozenset()] + (x^-1+2*x)*C[frozenset({9})]
iterating over frozenset({8})
G in L.order_ideal([F]): [frozenset(), frozenset({8})]
frozenset() 1 + 2*x^2
frozenset({8}) -x^-1 - 2*x
summing over frozenset({8}) gives -(1+2*x^2)*C[frozenset()] + (x^-1+2*x)*C[frozenset({8})]
iterating over frozenset({7})
G in L.order_ideal([F]): [frozenset(), frozenset({7})]
frozenset() 1 + 2*x^2
frozenset({7}) -x^-1 - 2*x
summing over frozenset({7}) gives -(1+2*x^2)*C[frozenset()] + (x^-1+2*x)*C[frozenset({7})]
iterating over frozenset()
G in L.order_ideal([F]): [frozenset()]
frozenset() -x^-2 - 12 - 5*x^2
summing over frozenset() gives (x^-2+12+5*x^2)*C[frozenset()]
iterating over frozenset({8, 9, 7})
G in L.order_ideal([F]): [frozenset(), frozenset({8}), frozenset({9}), frozenset({7}), frozenset({8, 9, 7})]
frozenset() 1 - 2*x^2

(x^-2+8+x^2)*C[frozenset()] + (x^-1+x)*C[frozenset({8})] + (x^-1+x)*C[frozenset({9})] + (x^-1+x)*C[frozenset({7})] + C[frozenset({8, 9, 7})]

In [201]:
def delta_diff_chow_conj_fixG(M, G, f):
    print("G:", G)
    ground_set = M.groundset()
    mdele = M.delete(e)
    sum = 0
    Fs = set(L.order_filter([G])).intersection(flat_div['S'])
    print("Fs:", Fs)
    for F in Fs:
        minor = M.delete(ground_set - set(F)).contract(G)
        chibar = chi(minor)(x^-2)
        poly1 = f(M.contract(F))(x^-2)
        poly2 = f(M.contract(F.union({e})))(x^-2)
        poly3 = f(mdele.contract(F))(x^-2)
        term = -P((poly1 + x^(-2)*poly2 - poly3)*chibar * x^(M.rank(ground_set) - M.rank(G)))
        print("flat", F, "gives:\n")
        print("f(M.contract(F))(x^-2) =", poly1)
        print("f(M.contract(F.union({e})))(x^-2) =", poly2)
        print("f(mdele.contract(F))(x^-2) =", poly3)
        print("chibar =", chibar)
        print("term =", term, "\n")
        sum += term
    
    assert x^(M.rank(ground_set) - M.rank(G) - 2)*f(M.contract(G.union({e})))(x^-2) == sum
    return sum

for G in flat_div['S']:
    print("--------------------------------")
    print(G,": ", delta_diff_chow_conj_fixG(M, G, chow_polynomial))
    print("--------------------------------")
    print("\n")


--------------------------------
G: frozenset({9})
Fs: {frozenset({9}), frozenset({8, 9, 7})}
flat frozenset({9}) gives:

f(M.contract(F))(x^-2) = x^-4 + 8*x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = x^-2 + 1
f(mdele.contract(F))(x^-2) = x^-4 + 7*x^-2 + 1
chibar = -1
term = x^-1 + 2*x 

flat frozenset({8, 9, 7}) gives:

f(M.contract(F))(x^-2) = x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = 1
f(mdele.contract(F))(x^-2) = x^-2 + 1
chibar = 1
term = -x 

frozenset({9}) :  x^-1 + x
--------------------------------


--------------------------------
G: frozenset({8})
Fs: {frozenset({8, 9, 7}), frozenset({8})}
flat frozenset({8, 9, 7}) gives:

f(M.contract(F))(x^-2) = x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = 1
f(mdele.contract(F))(x^-2) = x^-2 + 1
chibar = 1
term = -x 

flat frozenset({8}) gives:

f(M.contract(F))(x^-2) = x^-4 + 8*x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = x^-2 + 1
f(mdele.contract(F))(x^-2) = x^-4 + 7*x^-2 + 1
chibar = -1
term = x^-1 + 2*x 

frozenset({8}) :  x^-1 + x